In [1]:
import tensorflow as tf 
import numpy as np
import random

import matplotlib.pyplot as plt
from matplotlib import rcParams

tf.keras.config.set_floatx('float64')

#tf.config.run_functions_eagerly(True)

# Define the number of neuron per layer
NN = 5
#Define the number of layers 
NL = 3
#Define number of collocation points
N_points = 500
#
N_test = 100 # per time step

# Define the number of epochs
N_epoch = 10000

# # Define the manufactured solution
def u_sol(x, t):
    return tf.exp(-t) * tf.sin(x)

# # Compute the time derivative of u (u_t)
# def u_t(x, t):
#     return -tf.exp(-t) * tf.sin(x)

# # Compute the Laplacian of u (u_xx) in one dimension
# def u_xx(x, t):
#     return -tf.exp(-t) * tf.sin(x)

# Compute the source term f(x, t) = u_t - u_xx
def f_source(x,t):
    return -tf.exp(t)*tf.sin(x)*(np.pi**2-1)

def U_init(x):
    return tf.sin(x)

# PARA IMPONER CONDICIONES DE FRONTERA
class cutoff(tf.keras.layers.Layer):

    def __init__(self,**kwargs):
        super(cutoff,self).__init__()
    
    def call(self,inputs):
    
        x,u =inputs
        cut = x*(x-np.pi)
            
        return tf.einsum("ji,j->ji",cut,u) 


## Time discretization
delta_t = 0.0001
t_steps = 10

#Input dummy
X = tf.keras.layers.Input(shape = (1,))

#Define the first layer with dummy input
L1 = tf.keras.layers.Dense(units=NN, activation = 'tanh')(X)

#Feed the first layer to a generic layer for the number of layers, without the first and last layers
for l in range(NL - 2):
    L1 = tf.keras.layers.Dense(units=NN, activation = 'tanh')(L1)

#Last layer after being fed with the second to last layer (1 output)
L_last = tf.keras.layers.Dense(units=t_steps, activation = 'tanh',dtype='float64')(L1)

L_out = []
for i in range(t_steps):
    L0 = cutoff()([X,L_last[:,i]])
    L_out.append(L0)
    

#Assembly the model
U_model = tf.keras.Model(inputs = X, outputs = L_out, name = 'U_model')

#model summary
#U_model.summary()


2024-11-28 14:18:59.927672: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
#Define a class for the loss, we are going to call this class in every iteration, 
# evaluating this on the randomized sample, the target function and the output 
# of the network
###############################################################################
#Parameters for the class "loss": U_model --> Neural Network Architecture     #
#           N_points --> Number of Sampled points across the problem's domain #
#           f_source --> Source function on the PDE -Δu = f                   #
###############################################################################
class loss(tf.keras.layers.Layer):

    def __init__(self, U_model, N_points,f_source,N_test,t_steps,U_init,delta_t, **kwargs):
        super(loss,self).__init__()
        self.U_model = U_model
        self.N_points = N_points
        self.f_source = f_source
        self.N_test = N_test
        self.t_steps = t_steps
        self.U_init = U_init
        self.dt = delta_t

    #Compute the derivative of the neural network U_nn
    def  call(self,input):
        
        #Get a sample of points in the domain for the integration. 
        N_pointsreal = self.N_points #random.randint(self.N_points-1, self.N_points)
        
        S = tf.experimental.numpy.linspace(0.,np.pi,num=N_pointsreal, dtype='float64')
       
        with tf.GradientTape(persistent=True) as t1:       
            t1.watch(S)
            # Ojo: la salida de esto tf.transpose(U_model(xlist)) es tf.Tensor: shape=(tiempos,espacio)
            ueval = tf.squeeze(self.U_model(S)) # (tiempos,espacio)

            loss_out = 0
            for t in range(self.t_steps):
              
                #Compute a vector of the source function evaluated in the sample points & 
                F = self.f_source(S,tf.constant(t*self.dt,dtype=tf.float64))

                # Arma las funciones de base que usamos en la integral de la fuente
                G = tf.experimental.numpy.asarray([(1/i)*(np.sqrt(2)/np.pi)*tf.math.sin(i*S) 
                                                   for i in range(1,self.N_test+1)],dtype='float64')
        
                if t == 0:
                    U0 = self.U_init(S)
                    Integral_source = tf.einsum('i,ki->k',U0,G)
                else:
                    Integral_source = tf.einsum('i,ki->k',self.dt*F+ueval[t-1],G)

                # Arma el gradiente de las funciones de base que usamos en la integral maluca
                H = tf.experimental.numpy.asarray([(np.sqrt(2)/np.pi)*tf.cos(i*S) 
                                                   for i in range(1,self.N_test+1)],dtype='float64')
            
                u_x_tn = t1.gradient(ueval[t],S)
                
                Integral_Unn = tf.einsum('i,ki->k',u_x_tn,H)

                Integral_nueva = tf.einsum('i,ki->k',ueval[t],G)

                loss_out += tf.reduce_sum((Integral_nueva + self.dt*Integral_Unn - Integral_source)**2)
        
        del t1

        return loss_out



#Tell tensorlofw to recognize this class as a loss
def loss_model(U_model,N_points, f_source,N_test,t_steps,U_init,dt):
    Xl = tf.keras.layers.Input(shape = (1,),dtype='float64')
    loss_layer = loss(U_model, N_points, f_source,N_test,t_steps,U_init,dt)(Xl)
    loss_f = tf.keras.Model(inputs = Xl, outputs = loss_layer)
    return loss_f

def loss_value(x,y):
    return y

#Assemble the model
model_all = loss_model(U_model,N_points,f_source,N_test,t_steps,U_init,delta_t)
optimizer = tf.keras.optimizers.Adam(learning_rate = 10**-3)
model_all.compile(optimizer = optimizer, loss = loss_value)

#Train the model
history = model_all.fit(np.array([1.]),np.array([1.]),epochs = N_epoch)


Epoch 1/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 26s 26s/step - loss: 148966.4844
Epoch 2/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 143281.0938
Epoch 3/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 137712.3750
Epoch 4/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 132266.2344
Epoch 5/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 126947.3750
Epoch 6/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 121759.7578
Epoch 7/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 116707.7812
Epoch 8/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 111796.2812
Epoch 9/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 107030.2109
Epoch 10/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 102414.6016
Epoch 11/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 97954.5312
Epoch 12/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 93655.0781
Epoch 13/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 89521.1094
Epoch 14/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 85

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 19975.4238
Epoch 75/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 19638.8066
Epoch 76/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 19309.7578
Epoch 77/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 18988.0254
Epoch 78/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 18673.3730
Epoch 79/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 18365.5664
Epoch 80/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 18064.3848
Epoch 81/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 17769.6211
Epoch 82/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 17481.0781
Epoch 83/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 17198.5723
Epoch 84/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 16921.9316
Epoch 85/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 16650.9980
Epoch 86/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 16385.6270
Epoch 87/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 16125.6836
Epoch 8

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 6949.7549
Epoch 152/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 6879.9697
Epoch 153/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 6811.5488
Epoch 154/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 6744.4644
Epoch 155/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 6678.6909
Epoch 156/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 6614.2021
Epoch 157/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 6550.9727
Epoch 158/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 6488.9780
Epoch 159/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 6428.1943
Epoch 160/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 6368.5972
Epoch 161/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 6310.1636
Epoch 162/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 6252.8711
Epoch 163/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 6196.6973
Epoch 164/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 6141.6201
Epoch 16

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 4158.3433
Epoch 229/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 4142.9087
Epoch 230/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 4127.7603
Epoch 231/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 4112.8921
Epoch 232/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 4098.2974
Epoch 233/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 4083.9707
Epoch 234/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 4069.9055
Epoch 235/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 4056.0964
Epoch 236/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 4042.5378
Epoch 237/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 4029.2236
Epoch 238/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 4016.1484
Epoch 239/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 4003.3071
Epoch 240/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 3990.6941
Epoch 241/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 3978.3042
Epoch 24

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 3474.4253
Epoch 306/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 3469.1377
Epoch 307/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 3463.8899
Epoch 308/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 3458.6807
Epoch 309/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - loss: 3453.5090
Epoch 310/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 3448.3738
Epoch 311/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 3443.2742
Epoch 312/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 3438.2087
Epoch 313/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 3433.1770
Epoch 314/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 3428.1777
Epoch 315/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 3423.2100
Epoch 316/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 3418.2729
Epoch 317/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 3413.3657
Epoch 318/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 3408.4875
Epoch 31

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - loss: 179.7281
Epoch 1461/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 178.8923
Epoch 1462/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 178.0604
Epoch 1463/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 177.2323
Epoch 1464/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 176.4080
Epoch 1465/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 175.5875
Epoch 1466/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 174.7708
Epoch 1467/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 173.9579
Epoch 1468/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 173.1487
Epoch 1469/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 172.3432
Epoch 1470/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 171.5415
Epoch 1471/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 170.7435
Epoch 1472/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 169.9493
Epoch 1473/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 169.1587
Epoch 147

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 125.7702
Epoch 1538/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - loss: 125.1969
Epoch 1539/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 124.6266
Epoch 1540/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 124.0592
Epoch 1541/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 123.4948
Epoch 1542/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 122.9333
Epoch 1543/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 122.3748
Epoch 1544/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 121.8192
Epoch 1545/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 121.2664
Epoch 1546/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 120.7166
Epoch 1547/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 120.1697
Epoch 1548/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 119.6256
Epoch 1549/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 119.0844
Epoch 1550/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 118.5461
Epoch 155

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 89.5205
Epoch 1615/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 89.1442
Epoch 1616/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 88.7700
Epoch 1617/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 88.3979
Epoch 1618/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 88.0280
Epoch 1619/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 87.6601
Epoch 1620/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 87.2944
Epoch 1621/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 86.9308
Epoch 1622/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - loss: 86.5693
Epoch 1623/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 86.2098
Epoch 1624/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 85.8524
Epoch 1625/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 85.4971
Epoch 1626/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 85.1438
Epoch 1627/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 84.7925
Epoch 1628/10000
1/1 ━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 270ms/step - loss: 65.8814
Epoch 1693/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - loss: 65.6442
Epoch 1694/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - loss: 65.4085
Epoch 1695/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - loss: 65.1742
Epoch 1696/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 64.9413
Epoch 1697/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - loss: 64.7098
Epoch 1698/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 64.4797
Epoch 1699/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step - loss: 64.2511
Epoch 1700/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - loss: 64.0238
Epoch 1701/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - loss: 63.7979
Epoch 1702/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 63.5734
Epoch 1703/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 63.3503
Epoch 1704/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 63.1285
Epoch 1705/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 62.9081
Epoch 1706/10000


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 51.1459
Epoch 1771/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 50.9995
Epoch 1772/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 50.8540
Epoch 1773/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 50.7094
Epoch 1774/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 50.5658
Epoch 1775/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 50.4230
Epoch 1776/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 50.2811
Epoch 1777/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 50.1401
Epoch 1778/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 50.0000
Epoch 1779/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 49.8607
Epoch 1780/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 49.7223
Epoch 1781/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 49.5848
Epoch 1782/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 49.4482
Epoch 1783/10000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 49.3123
Epoch 1784/10000
1/1 ━━

In [ ]:
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 18
rcParams['legend.fontsize'] = 17
rcParams['mathtext.fontset'] = 'cm' 
rcParams['axes.labelsize'] = 19


# Generate a list of x values for visualization
xlist = tf.experimental.numpy.linspace(0.,np.pi,num=1000, dtype='float32')

## ---------
# Graph the Solution
## ---------

fig, ax = plt.subplots()
uu = tf.squeeze(U_model(xlist))

t = 10
# Plot the approximate solution obtained from the trained model
plt.plot(xlist, uu[t-1], color='b')
#plt.plot(xlist, tf.exp(-delta_t) * tf.sin(xlist), color='m')
plt.plot(xlist, tf.exp(-delta_t*10) * tf.sin(xlist), color='g')

plt.legend(['$u_{NN}$', '$u$'])

ax.grid(which = 'both', axis = 'both', linestyle = ':', color = 'gray')
plt.tight_layout()
plt.title('Solucion')
plt.show()


Loss_rel = history.history['loss'] 
fig, ax = plt.subplots()
# Plot the approximate solution obtained from the trained model
plt.plot(Loss_rel, color='r')


ax.set_xscale('log')
ax.set_yscale('log')

ax.grid(which = 'major', axis = 'both', linestyle = ':', color = 'gray')
plt.tight_layout()
plt.title('Loss')
plt.show()

